# <span style="color:orange">Appliquer le code du Baromètre de la Science Ouverte aux données de l'UCA</span>

Une fois que l'on dispose de sa liste de DOI nettoyée (voir le notebook "nettoyage_donnees"), il est à présent temps d'appliquer le code du Baromètre national de la Science Ouverte aux données de son propre établissement. Il suffit de remplacer "lorraine" par le nom de son établissement à l'intérieur du code.

## <span style="color:orange">Importer les scripts</span>

Adresse :  https://github.com/MinistereSupRecherche/bso

Copier dans son propre répertoire, à la racine, les fichiers "scanr_utils.py" et "unpwaywall_utils.py". Ce n'est pas une faute de frappe, le fichier s'appelle bien "unpwaywall".

In [1]:
from scanr_utils import *
from unpwaywall_utils import *

## <span style="color:orange">Importer dans le notebook les données préalablement traitées</span>

In [2]:
with open("Data/outputs/doi_ready.csv") as f:
    doi_str = f.read()

In [3]:
doi_df = pd.read_csv("Data/outputs/doi_ready.csv",sep=",")

In [4]:
doi_df

,doi
0,10.1016/j.bmc.2017.08.043
1,10.1007/jhep06(2017)047 10.1007/jhep10(2017)068
2,10.1007/jhep05(2017)158
3,10.1016/j.mednuc.2017.02.061
4,10.15122/isbn.978-2-406-06967-6.p.0177
...,...
4649,10.1182/blood-2018-99-119656
4650,10.1017/s0142716418000346
4651,10.1016/s1474-4422(18)30027-9
4652,10.1123/jpah.2018-0472


## <span style="color:orange">Enrichissement avec l'API Unpaywall</span>

On ajoute à la liste de DOI constituée précédemment toutes les informations comprises dans l'API Unpaywall : titre, année, statut d'open access... <span style="color:red">**Attention, le processus peut prendre plusieurs heures la première fois que l'on exécute ce code.**</span>

In [5]:
publis_oa_status = enrich_with_upw_status(doi_df)
publis_oa_status
publis_oa_status.to_csv("Data/outputs/publis_oa_status.csv",index=False)

4654 publications
0 %, 1 %, 2 %, 3 %, 4 %, 5 %, 6 %, 8 %, 9 %, 10 %, 11 %, 12 %, 13 %, 14 %, 15 %, 16 %, 17 %, 18 %, 19 %, 20 %, 21 %, 23 %, 24 %, 25 %, 26 %, 27 %, 28 %, 29 %, 30 %, 31 %, 32 %, 33 %, 34 %, 35 %, 37 %, 38 %, 39 %, 40 %, 41 %, 42 %, 43 %, 44 %, 45 %, 46 %, 47 %, 48 %, 49 %, 50 %, 52 %, 53 %, 54 %, 55 %, 56 %, 57 %, 58 %, 59 %, 60 %, 61 %, 62 %, 63 %, 64 %, 66 %, 67 %, 68 %, 69 %, 70 %, 71 %, 72 %, 73 %, 74 %, 75 %, 76 %, 77 %, 78 %, 80 %, 81 %, 82 %, 83 %, 84 %, 85 %, 86 %, 87 %, 88 %, 89 %, 90 %, 91 %, 92 %, 93 %, 95 %, 96 %, 97 %, 98 %, 99 %, 100 %, 

Capturer le résultat pour ne pas avoir à refaire ce travail plusieurs fois :

In [6]:
publis_oa_status = pd.read_csv("Data/outputs/publis_oa_status.csv")

Avoir le fichier en format Excel et libre office lisible par les humains :

In [7]:
publis_oa_status.to_excel("Data/outputs/publis_oa_status.xls",index=False)
publis_oa_status.to_csv("Data/outputs/publis_oa_status.ods",index=False)

## <span style="color:orange">Ajouter à la DataFrame le champ "scientific field" pour ajouter le domaine pour chaque DOI grâce au jeu de données du Baromètre</soan>

Le jeu de données est disponible à cette adresse : https://data.enseignementsup-recherche.gouv.fr/explore/dataset/open-access-monitor-france/export/?disjunctive.oa_host_type&disjunctive.year

<span style="color:red">**Il faut le télécharger chaque année et le placer au même endroit (Barometre_SO_lorraine -> raw) pour mettre à jour les données.**</span>

### Lire le jeu de données du Baromètre national et le stocker dans une variable

In [8]:
with open("Data/raw/open-access-monitor-france.csv", encoding='utf-8') as f:
    open_access_monitor_france_str = f.read()

In [9]:
open_access_monitor_france_df = pd.read_csv("Data/raw/open-access-monitor-france.csv", encoding='utf-8',sep=";")

### Fusionner la DataFrame UCA et le jeu de données du Baromètre sur le critère du DOI pour ajouter le champ "scientific field"

In [10]:
df_complete = pd.merge(publis_oa_status, open_access_monitor_france_df[['doi', 'scientific_field']], how="left", on="doi")
df_complete['scientific_field'].fillna('unknown', inplace = True)

In [11]:
df_complete

,doi,oa_type,is_oa,title,published_date,published_year,genre,journal_is_in_doaj,journal_issns,journal_name,publisher,scientific_field
0,10.1016/j.bmc.2017.08.043,closed,False,Linker structure-activity relationships in flu...,2017-10-01,2017.0,journal-article,False,0968-0896,Bioorganic & Medicinal Chemistry,Elsevier BV,Medical research
1,10.1007/jhep06(2017)047 10.1007/jhep10(2017)068,publisher;repository,True,Measurement of the J/ψ pair production cross-s...,2017-06-01,2017.0,journal-article,True,1029-8479,Journal of High Energy Physics,Springer Science and Business Media LLC,unknown
2,10.1007/jhep05(2017)158,publisher;repository,True,Search for the B s 0 → η′ϕ decay,2017-05-01,2017.0,journal-article,True,1029-8479,Journal of High Energy Physics,Springer Science and Business Media LLC,"Physical sciences, Astronomy"
3,10.1016/j.mednuc.2017.02.061,closed,False,Radiomarquage du NTP 15-5 ciblant les protéogl...,2017-05-01,2017.0,journal-article,False,0928-1258,Médecine Nucléaire,Elsevier BV,Medical research
4,10.15122/isbn.978-2-406-06967-6.p.0177,closed,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Humanities
...,...,...,...,...,...,...,...,...,...,...,...,...
4649,10.1182/blood-2018-99-119656,publisher,True,The Management of Cardiovascular Diseases in P...,2018-11-29,2018.0,journal-article,False,"0006-4971,1528-0020",Blood,American Society of Hematology,Medical research
4650,10.1017/s0142716418000346,closed,False,The effect of morphosyntactic training on mult...,2018-10-10,2018.0,journal-article,False,"0142-7164,1469-1817",Applied Psycholinguistics,Cambridge University Press (CUP),Humanities
4651,10.1016/s1474-4422(18)30027-9,closed,False,Use of brain diffusion tensor imaging for the ...,2018-04-01,2018.0,journal-article,False,1474-4422,The Lancet Neurology,Elsevier BV,Medical research
4652,10.1123/jpah.2018-0472,publisher;repository,True,Global Matrix 3.0 Physical Activity Report Car...,2018-01-02,2018.0,journal-article,False,"1543-3080,1543-5474",Journal of Physical Activity and Health,Human Kinetics,Medical research


Les publications de l'Université de Lorraine ont été enrichies par l'ajout du champ "scientific field" produit par le Ministère. En revanche, les données lorraines couvrent la période 2016-2019, alors que le Baromètre s'arrête en 2018 : l'information du domaine disciplinaire sera donc absente pour l'année 2019. 

In [12]:
df_complete.to_csv("Data/outputs/publis_completes.csv",index=False)

<span style="color:green">**La liste de DOI enrichie par les informations présentes dans l'API Unpaywall et par le code du Ministère permettant de connaître la discipline a été stockée sous forme de fichier CSV dans le dossier Data > outputs.**</span>

Avoir le fichier en format Excel lisible par les humains :

In [13]:
df_complete.to_excel("Data/outputs/publis_completes.xls",index=False)
df_complete.to_csv("Data/outputs/publis_oa_status.ods",index=False)                     

A partir de ce fichier, il est très simple de construire des requêtes. Exemple : combien de publications auprès de l'éditeur Wiley ont été recensées dans le jeu de données ?

In [16]:
df_complete.loc[df_complete['publisher'] == "Wiley",:]

,doi,oa_type,is_oa,title,published_date,published_year,genre,journal_is_in_doaj,journal_issns,journal_name,publisher,scientific_field
3,10.1002/1878-0261.12191,publisher;repository,True,DNA methylation profiling reveals a pathologic...,2018-04-27,2018.0,journal-article,True,"1574-7891,1878-0261",Molecular Oncology,Wiley,Medical research
5,10.1002/2211-5463.12415,publisher;repository,True,Comparative metabolomics and glycolysis enzyme...,2018-04-17,2018.0,journal-article,True,"2211-5463,2211-5463",FEBS Open Bio,Wiley,Biology (fond.)
7,10.1002/acn3.648,publisher;repository,True,Deep brain stimulation in five patients with s...,2018-09-26,2018.0,journal-article,True,2328-9503,Annals of Clinical and Translational Neurology,Wiley,Medical research
8,10.1002/adfm.201703868,closed,False,Tailoring Hybrid Layered Double Hydroxides for...,2017-11-17,2017.0,journal-article,False,1616-301X,Advanced Functional Materials,Wiley,"Physical sciences, Astronomy"
9,10.1002/ajh.24990,publisher;repository,True,“Double-hit” chronic lymphocytic leukemia: An ...,2017-12-18,2017.0,journal-article,False,0361-8609,American Journal of Hematology,Wiley,Medical research
...,...,...,...,...,...,...,...,...,...,...,...,...
2373,10.1111/1756-185x.13375,publisher;repository,True,"Retention rates of adalimumab, etanercept, and...",2018-08-30,2018.0,journal-article,False,"1756-1841,1756-185X",International Journal of Rheumatic Diseases,Wiley,Medical research
2385,10.1002/nag.2837,publisher,True,Bounding surface-based modeling of compacted s...,2018-07-05,2018.0,journal-article,False,0363-9061,International Journal for Numerical and Analyt...,Wiley,Engineering
2388,10.1002/ajh.25133,publisher,True,HLA-DRB3/4/5 mismatches are associated with in...,2018-06-20,2018.0,journal-article,False,0361-8609,American Journal of Hematology,Wiley,Medical research
2400,10.1111/jasp.12530,closed,False,Perceived acculturation preferences of minorit...,2018-07-31,2018.0,journal-article,False,0021-9029,Journal of Applied Social Psychology,Wiley,Biology (fond.)
